In [ ]:
!pip3 install pandas geopandas shapely fiona geopy

In [33]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import fiona
# import geopy
from geopy.geocoders import Nominatim
import requests
import re
import json

In [2]:
banksCA = pd.read_csv("./data/processed/OFFICES2_CALIF_geocodio.csv")
banksAZNVOR = pd.read_csv("./data/processed/OFFICES2_AZ_NV_OR_geocodio.csv")

/Users/work/.pyenv/versions/3.9.6/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (35,48,54,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
banksCA.columns.values.tolist()

['ADDRESS',
 'ADDRESS_FULL',
 'BKCLASS',
 'CBSA',
 'CBSA_DIV',
 'CBSA_DIV_FLG',
 'CBSA_DIV_NO',
 'CBSA_METRO',
 'CBSA_METRO_FLG',
 'CBSA_METRO_NAME',
 'CBSA_MICRO_FLG',
 'CBSA_NO',
 'CERT',
 'CITY',
 'COUNTY',
 'CSA',
 'CSA_FLG',
 'CSA_NO',
 'ESTYMD',
 'FI_UNINUM',
 'MAINOFF',
 'NAME',
 'OFFNAME',
 'OFFNUM',
 'RUNDATE',
 'SERVTYPE',
 'STALP',
 'STCNTY',
 'STNAME',
 'UNINUM',
 'ZIP',
 'Latitude',
 'Longitude',
 'Accuracy Score',
 'Accuracy Type',
 'Number',
 'Street',
 'Unit Type',
 'Unit Number',
 'City',
 'State',
 'County',
 'Zip',
 'Country',
 'Source',
 'ZIP+4 Range Start',
 'ZIP+4 Range End (or blank if no range)',
 'ZIP 9 Range Start',
 'ZIP 9 Range End (or blank if no range)',
 'Record Type Code',
 'Record Type Description',
 'Carrier Route ID',
 'Carrier Route Description',
 'Building or Firm Name',
 'Government Building Code',
 'Government Building Description',
 'Facility Code',
 'Facility Description',
 'City Delivery?',
 'Valid delivery area?',
 'Exact Match?',
 'Census Yea

In [4]:
banksAZNVOR.columns.values.tolist()

['ADDRESS',
 'ADDRESS_FULL',
 'BKCLASS',
 'CBSA',
 'CBSA_DIV',
 'CBSA_DIV_FLG',
 'CBSA_DIV_NO',
 'CBSA_METRO',
 'CBSA_METRO_FLG',
 'CBSA_METRO_NAME',
 'CBSA_MICRO_FLG',
 'CBSA_NO',
 'CERT',
 'CITY',
 'COUNTY',
 'CSA',
 'CSA_FLG',
 'CSA_NO',
 'ESTYMD',
 'FI_UNINUM',
 'MAINOFF',
 'NAME',
 'OFFNAME',
 'OFFNUM',
 'RUNDATE',
 'SERVTYPE',
 'STALP',
 'STCNTY',
 'STNAME',
 'UNINUM',
 'ZIP',
 'Latitude',
 'Longitude',
 'Accuracy Score',
 'Accuracy Type',
 'Number',
 'Street',
 'Unit Type',
 'Unit Number',
 'City',
 'State',
 'County',
 'Zip',
 'Country',
 'Source',
 'Census Year',
 'State FIPS',
 'County FIPS',
 'Place Name',
 'Place FIPS',
 'Census Tract Code',
 'Census Block Code',
 'Census Block Group',
 'Full FIPS (block)',
 'Full FIPS (tract)',
 'Metro/Micro Statistical Area Name',
 'Metro/Micro Statistical Area Code',
 'Metro/Micro Statistical Area Type',
 'Combined Statistical Area Name',
 'Combined Statistical Area Code',
 'Metropolitan Division Area Name',
 'Metropolitan Division Area 

In [3]:
fields = []
for field in banksCA.columns.values.tolist():
    if field not in banksAZNVOR.columns.values.tolist():
        fields.append(field)
print(fields)

['ZIP+4 Range Start', 'ZIP+4 Range End (or blank if no range)', 'ZIP 9 Range Start', 'ZIP 9 Range End (or blank if no range)', 'Record Type Code', 'Record Type Description', 'Carrier Route ID', 'Carrier Route Description', 'Building or Firm Name', 'Government Building Code', 'Government Building Description', 'Facility Code', 'Facility Description', 'City Delivery?', 'Valid delivery area?', 'Exact Match?']


In [4]:
banksCA.drop(fields, axis=1, inplace=True)
banksCA

,ADDRESS,ADDRESS_FULL,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,...,ACS Demographics/Race and ethnicity/Hispanic or Latino: Some other race alone/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Percentage,"ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Value","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Margin of error","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Percentage"
0,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Century ...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.000,0.0,12.0,0.000
1,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Los Ange...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.000,0.0,12.0,0.000
2,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.121,246.0,334.0,0.463,246.0,334.0,1.000,0.0,12.0,0.000
3,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.121,246.0,334.0,0.463,246.0,334.0,1.000,0.0,12.0,0.000
4,"255 California Street, Suite 600","255 California Street, Suite 600, San Francisc...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.493,10.0,15.0,0.137,10.0,15.0,1.000,0.0,12.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6327,6 Hutton Centre Drive Suite 100,"6 Hutton Centre Drive Suite 100, Santa Ana, CA...",NM,"Los Angeles-Long Beach-Anaheim, CA","Anaheim-Santa Ana-Irvine, CA",1,11244,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.226,84.0,65.0,0.032,36.0,35.0,0.429,48.0,43.0,0.571
6328,750 B Street Suite 3110,"750 B Street Suite 3110, San Diego, CA 92101",NM,"San Diego-Chula Vista-Carlsbad, CA",NaN,0,0,41740,1,"San Diego-Chula Vista-Carlsbad, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.000,0.0,12.0,0.000
6329,"4675 Macarthur Court, Suite 1600","4675 Macarthur Court, Suite 1600, Newport Beac...",NM,"Los Angeles-Long Beach-Anaheim, CA","Anaheim-Santa Ana-Irvine, CA",1,11244,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.365,30.0,27.0,0.030,19.0,30.0,0.633,11.0,18.0,0.367
6330,"520 Newport Center Drive, Suite 700","520 Newport Center Drive, Suite 700, Newport B...",N,"Los Angeles-Long Beach-Anaheim, CA","Anaheim-Santa Ana-Irvine, CA",1,11244,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.000,0.0,12.0,0.000


In [5]:
banksDF = pd.concat([banksCA, banksAZNVOR])

In [6]:
banksDF.reset_index(drop=True, inplace=True)
banksDF

,ADDRESS,ADDRESS_FULL,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,...,ACS Demographics/Race and ethnicity/Hispanic or Latino: Some other race alone/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Percentage,"ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Value","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Margin of error","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Percentage"
0,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Century ...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0
1,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Los Ange...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0
2,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.121,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0
3,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.121,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0
4,"255 California Street, Suite 600","255 California Street, Suite 600, San Francisc...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,0.493,10.0,15.0,0.137,10.0,15.0,1.0,0.0,12.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,5370 Kietzke Lane,"5370 Kietzke Lane, Reno, NV 89511",N,"Reno, NV",NaN,0,0,39900,1,"Reno, NV",...,0.234,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0
8695,2405 West Burnside Street,"2405 West Burnside Street, Portland, OR 97210",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.149,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0
8696,947 Sw Broadway,"947 Sw Broadway, Portland, OR 97205",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.000,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0
8697,"2360 Corporate Circle, Suite 400","2360 Corporate Circle, Suite 400, Henderson, N...",SB,"Las Vegas-Henderson-Paradise, NV",NaN,0,0,29820,1,"Las Vegas-Henderson-Paradise, NV",...,0.000,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0


In [7]:
banksDF['geometry'] = banksDF.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)


In [12]:
len(banksDF['geometry'])

8699

In [8]:
banksDF

,ADDRESS,ADDRESS_FULL,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,...,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Percentage,"ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Value","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Margin of error","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Percentage",geometry
0,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Century ...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-118.416419 34.058859)
1,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Los Ange...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-118.416419 34.058859)
2,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0,POINT (-122.395607 37.788001)
3,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0,POINT (-122.395607 37.788001)
4,"255 California Street, Suite 600","255 California Street, Suite 600, San Francisc...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,10.0,15.0,0.137,10.0,15.0,1.0,0.0,12.0,0.0,POINT (-122.399668 37.793118)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,5370 Kietzke Lane,"5370 Kietzke Lane, Reno, NV 89511",N,"Reno, NV",NaN,0,0,39900,1,"Reno, NV",...,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0,POINT (-119.790735 39.469506)
8695,2405 West Burnside Street,"2405 West Burnside Street, Portland, OR 97210",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-122.700313 45.524249)
8696,947 Sw Broadway,"947 Sw Broadway, Portland, OR 97205",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-122.680924 45.517641)
8697,"2360 Corporate Circle, Suite 400","2360 Corporate Circle, Suite 400, Henderson, N...",SB,"Las Vegas-Henderson-Paradise, NV",NaN,0,0,29820,1,"Las Vegas-Henderson-Paradise, NV",...,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0,POINT (-115.088583 36.025051)


In [9]:
banksGDF = gpd.GeoDataFrame(banksDF, crs="EPSG:4326", geometry=banksDF.geometry)

In [10]:
banksGDF

,ADDRESS,ADDRESS_FULL,BKCLASS,CBSA,CBSA_DIV,CBSA_DIV_FLG,CBSA_DIV_NO,CBSA_METRO,CBSA_METRO_FLG,CBSA_METRO_NAME,...,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races/Percentage,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Value,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Margin of error,ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races including Some other race/Percentage,"ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Value","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Margin of error","ACS Demographics/Race and ethnicity/Hispanic or Latino: Two or more races: Two races excluding Some other race, and three or more races/Percentage",geometry
0,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Century ...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-118.41642 34.05886)
1,"1999 Avenue Of The Stars, Suite 1100","1999 Avenue Of The Stars, Suite 1100, Los Ange...",NM,"Los Angeles-Long Beach-Anaheim, CA","Los Angeles-Long Beach-Glendale, CA",1,31084,31080,1,"Los Angeles-Long Beach-Anaheim, CA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-118.41642 34.05886)
2,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0,POINT (-122.39561 37.78800)
3,"One Embarcadero Center, Suite 3840","One Embarcadero Center, Suite 3840, San Franci...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,246.0,334.0,0.463,246.0,334.0,1.0,0.0,12.0,0.0,POINT (-122.39561 37.78800)
4,"255 California Street, Suite 600","255 California Street, Suite 600, San Francisc...",NM,"San Francisco-Oakland-Berkeley, CA","San Francisco-San Mateo-Redwood City, CA",1,41884,41860,1,"San Francisco-Oakland-Berkeley, CA",...,10.0,15.0,0.137,10.0,15.0,1.0,0.0,12.0,0.0,POINT (-122.39967 37.79312)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8694,5370 Kietzke Lane,"5370 Kietzke Lane, Reno, NV 89511",N,"Reno, NV",NaN,0,0,39900,1,"Reno, NV",...,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0,POINT (-119.79073 39.46951)
8695,2405 West Burnside Street,"2405 West Burnside Street, Portland, OR 97210",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-122.70031 45.52425)
8696,947 Sw Broadway,"947 Sw Broadway, Portland, OR 97205",NM,"Portland-Vancouver-Hillsboro, OR-WA",NaN,0,0,38900,1,"Portland-Vancouver-Hillsboro, OR-WA",...,0.0,12.0,0.000,0.0,12.0,0.0,0.0,12.0,0.0,POINT (-122.68092 45.51764)
8697,"2360 Corporate Circle, Suite 400","2360 Corporate Circle, Suite 400, Henderson, N...",SB,"Las Vegas-Henderson-Paradise, NV",NaN,0,0,29820,1,"Las Vegas-Henderson-Paradise, NV",...,0.0,13.0,0.000,0.0,13.0,0.0,0.0,13.0,0.0,POINT (-115.08858 36.02505)


In [16]:
banksGDF.to_file("data/processed/OFFICES2_CA_AZ_NV_OR_geocodio.geojson", driver="GeoJSON")

In [13]:
banksGDF.geometry.head()

0    POINT (-118.41642 34.05886)
1    POINT (-118.41642 34.05886)
2    POINT (-122.39561 37.78800)
3    POINT (-122.39561 37.78800)
4    POINT (-122.39967 37.79312)
Name: geometry, dtype: geometry

In [ ]:
import requests

body = {"locations":[[8.681495,49.41461],[8.686507,49.41943]],"range":[300,200]}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': 'your-api-key',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/isochrones/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

In [39]:
coords = []

for i in banksGDF.geometry.head(2):
    # print(re.sub("POINT", "", i))
    # print(type(i.x))
    xy = [i.x,i.y]
    coords.append(xy)
print(coords)

[[-118.416419, 34.058859], [-118.416419, 34.058859]]


In [41]:
body = {
    'locations': coords,
    'range': [300,200]
    }

# y = json.dumps(body)

# print(y)
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': 'your-api-key',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('http://localhost:8080/ors/v2/isochrones/driving-car', json=body, headers=headers)

print(call.status_code, call.reason)
print(call.text)

500 
{"error":{"code":3099,"message":"Unable to build an isochrone map."},"info":{"engine":{"version":"6.6.1","build_date":"2021-11-11T07:53:01Z"},"timestamp":1637105014338}}
